In [1]:
import fastai.text
import pandas as pd

In [2]:
from nltk import tokenize

In [3]:
%run ../preprocessing.py

In [4]:
replace_with_rules

<function __main__.replace_with_rules(s, rules)>

In [5]:
!ls ../data

file.csv  nlp_github_repos.json


In [6]:
raw_df = pd.read_json('../data/nlp_github_repos.json', lines=True)
raw_df.head()

,repo_name,path,content,language
0,mortardata/mortar-etl-redshift,README.md,# Mortar ETL Pipeline for Redshift\n\nA custom...,"[{'name': 'PigLatin', 'bytes': '4890'}, {'name..."
1,tsubery/sidekiq-unique-jobs,README.md,# SidekiqUniqueJobs [![Build Status](https://t...,"[{'name': 'Ruby', 'bytes': '41751'}]"
2,ovaskevich/svids,README.md,# svids\nIntrusion Detection System - An appli...,"[{'name': 'C', 'bytes': '3755'}, {'name': 'Mak..."
3,kfirg77/kDebuger,README.md.orig,kDebuger\n========\n\nShow information about t...,"[{'name': 'CSS', 'bytes': '27336'}, {'name': '..."
4,wdi-hk-sep-2014/PepperLunchClone,README.rdoc,== README\n\nThis README would normally docume...,"[{'name': 'CSS', 'bytes': '2444'}, {'name': 'C..."


In [7]:
test_df = raw_df.copy()
test_df = test_df[~test_df['content'].isna()]
test_df = test_df.iloc[10:20]

In [8]:
test_df['cleaned_content']  = test_df['content'].apply(markdown_to_text)

/home/kuba/Projects/github_search/preprocessing.py:54: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 54 of the file /home/kuba/Projects/github_search/preprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  return BeautifulSoup(html).get_text()


In [9]:
for r in test_df['content']:
    print(r)
    print('*' * 50)

Bitcoin Core integration/staging tree

[![Build Status](https://travis-ci.org/bitcoin/bitcoin.svg?branch=master)](https://travis-ci.org/bitcoin/bitcoin)

https://www.bitcoin.org

What is Bitcoin?
----------------

Bitcoin is an experimental new digital currency that enables instant payments to
anyone, anywhere in the world. Bitcoin uses peer-to-peer technology to operate
with no central authority: managing transactions and issuing money are carried
out collectively by the network. Bitcoin Core is the name of open source
software which enables the use of this currency.

For more information, as well as an immediately useable, binary version of
the Bitcoin Core software, see https://www.bitcoin.org/en/download.

License
-------

Bitcoin Core is released under the terms of the MIT license. See [COPYING](COPYING) for more
information or see http://opensource.org/licenses/MIT.

Development process
-------------------

Developers work in their own trees, then submit pull requests when they t

In [10]:
for r in test_df['cleaned_content']:
    print(r)
    print('*' * 50)

Bitcoin Core integration/staging tree

xxurl 
What is Bitcoin?
Bitcoin is an experimental new digital currency that enables instant payments to
anyone, anywhere in the world. Bitcoin uses peer-to-peer technology to operate
with no central authority: managing transactions and issuing money are carried
out collectively by the network. Bitcoin Core is the name of open source
software which enables the use of this currency.
For more information, as well as an immediately useable, binary version of
the Bitcoin Core software, see xxurl 
License
Bitcoin Core is released under the terms of the MIT license. See COPYING for more
information or see xxurl 
Development process
Developers work in their own trees, then submit pull requests when they think
their feature or bug fix is ready.
If it is a simple/trivial/non-controversial change, then one of the Bitcoin
development team members simply pulls it.
If it is a more complicated or potentially controversial change, then the patch
submitter will b

# HERE BE DRAGONS

In [11]:
markdown_tokenizer = fastai.text.Tokenizer(
    tok_func=tok_fn,
    pre_rules=[],
    post_rules=[],
    special_cases=[],
    n_cpus=12)

In [12]:
raw_df['languages'] = raw_df['language'].apply(lambda ds: [d['name'] for d in ds])

In [13]:
selected_langs = ['python', 'r', 'matlab', 'julia', 'c++', 'java', 'scala']
df = raw_df[raw_df['languages'].apply(lambda langs: any([lang.lower() in selected_langs for lang in langs]))]
df = df.drop(['language'], axis=1)
df = df[(~df['content'].isna())]
df = df[df['content'].str.split().apply(len) > 25]
df = df[(df['content'].apply(itemgetter(0)) != '<') & (df['content'].apply(itemgetter(-1)) != '>')]
df['content'] = df['content'].str.replace(r'#+', 'xxhashtag')

In [14]:
df['content'] = df['content'].str.replace(r'[0-9]+', 'xxnumber')

In [16]:
n_examples =  10 * 10 ** 3
lm_df = df[['repo_name', 'languages', 'content']][:n_examples]

In [17]:
lm_df = lm_df.dropna()

In [18]:
lm_df[['content']].to_csv('github_repos_lm_text_small.csv')

In [19]:
lm_df.index = pd.RangeIndex(len(lm_df))

In [21]:
import tqdm

extracted_content = pd.Series([tokenize_markdown(md_string) for md_string in tqdm.tqdm(lm_df['content'])])
lm_df['text'] = extracted_content.apply(' '.join)
lm_df = lm_df[(~lm_df['text'].isna()) & (lm_df['text'].apply(len) > 0)]
lm_df[['text']].to_csv('github_repos_lm_text.csv')

100%|██████████| 10000/10000 [00:45<00:00, 219.97it/s]


### Load to FastAI api

In [23]:
%%time
data_lm = fastai.text.TextLMDataBunch.from_csv(
    '', 'github_repos_lm_text.csv'
)
data_lm.save('data_lm_export.pkl')

CPU times: user 1.15 s, sys: 257 ms, total: 1.4 s
Wall time: 4.69 s


In [24]:
bs = 64

In [26]:
data_lm = fastai.text.load_data('', 'data_lm_export.pkl', bs=bs, bptt=50)

In [27]:
data_lm

TextLMDataBunch;

Train: LabelList (7999 items)
x: LMTextList
xxbos xxhashtag xxup xxunk - xxmaj edison - xxmaj python i found a lot of the existing code for xxmaj intel xxmaj edison and xxmaj sparkfun xxmaj edison xxmaj blocks to be confusing . xxmaj this repo attempts to provide simplified get - it - working python code for : xxhashtag xxmaj sparkfun xxmaj edison xxmaj blocks xxnumberdof xxup imu xxhashtag xxmaj sparkfun xxmaj edison xxmaj blocks xxup adc xxmaj none of these would happen without the community at large , so please see each file for license and credit .,xxbos xxmaj unbound xxup readme . / configure & & make & & make install xxmaj you can use libevent if you want . libevent is useful when using many ( xxnumber ) outgoing ports . xxmaj by default max xxnumber ports are opened at the same time and the builtin alternative is equally capable and a little faster . xxmaj more detailed xxup readme , xxup readme . svn , xxup readme . tests in doc directory manual pages can be f